# OSeMOSYS-PLEXOS global model
## Powerplant data

### Import modules

In [ ]:
import pandas as pd
import numpy as np

### Import data files

In [ ]:
df = pd.read_csv(r'OPG_powerplant_data_19-03-2020.csv', 
                 encoding='latin-1', 
                 low_memory=False)
df_dict = pd.read_csv(r'OPG_memberships_19-03-2020.csv')
df_dict = df_dict[df_dict['parent_class']=='Generator'].rename({'parent_object':'powerplant'}, axis=1)

In [ ]:
gen_cols_1 = ['child_class', 'child_object', 'property', 'value'] 
df_gen = df[gen_cols_1]
df_gen = df_gen[df_gen['child_class']=='Generator']
df_gen.rename(columns={'child_object':'powerplant'}, 
              inplace=True)
df_gen.drop('child_class', 
            axis=1, 
            inplace=True)
df_gen = pd.pivot_table(df_gen, 
                        index='powerplant', 
                        columns='property', 
                        values='value', 
                        aggfunc=np.sum, 
                        fill_value=0)
df_gen['total_capacity'] = (df_gen['Max Capacity'].astype(float))*(df_gen['Units'].astype(int))

In [ ]:
gen_cols_2 = ['Commission Date', 'Heat Rate', 'Max Capacity', 'total_capacity']
df_gen_2 = df_gen[gen_cols_2]

## Compile dataframe with powerplants, nodes, and fuels
df_dict_fuel = df_dict[df_dict['collection']=='Fuels']
df_dict_fuel = df_dict_fuel[['powerplant', 'child_object']]
df_dict_nodes = df_dict[df_dict['collection']=='Nodes']
df_dict_nodes = df_dict_nodes[['powerplant', 'child_object']]
df_dict_2 = pd.merge(df_dict_fuel, df_dict_nodes, how='outer', on='powerplant')

## Merge original generator dataframe with nodes and fuels
df_gen_2 = pd.merge(df_gen_2, df_dict_2, how='outer', on='powerplant')
df_gen_2.rename({'child_object_x':'fuel','child_object_y':'node'}, axis=1, inplace=True)

## Extract start year from Commission Date 
df_gen_2['Commission Date'] = pd.to_datetime(df_gen_2['Commission Date'])
df_gen_2['start_year'] = df_gen_2['Commission Date'].dt.year
df_gen_2.drop('Commission Date', axis=1, inplace=True)

## Calculate efficiency from heat rate. Units of heat rate in MJ/kWh
df_gen_2['efficiency'] = 3.6/df_gen_2['Heat Rate'].astype(float)
df_gen_2.drop('Heat Rate', axis=1, inplace=True)

## Calcluate years of operation from start year until 2015
df_gen_2['years_of_operation'] = 2015 - df_gen_2['start_year']

## Fix blank spaces in 'fuels' columns. Appearing for 'Oil' powerplants in certain countries
df_gen_2.loc[df_gen_2['fuel'].isna(), 'fuel'] = df_gen_2['node'].str.split('-').str[:2].str.join('-') + ' ' + df_gen_2['powerplant'].str.split('_', expand=True)[1] 


In [ ]:
## Create column for technology
df_gen_2['technology'] = df_gen_2['powerplant'].str.split('_').str[1]
df_gen_2['technology'] = df_gen_2['technology'].str.title()

df_gen_2.loc[(df_gen_2['technology']=='Gas') & (df_gen_2['Max Capacity'].astype(float) > 130),'technology'] = 'Gas-CCGT' 
df_gen_2.loc[(df_gen_2['technology']=='Gas') & (df_gen_2['Max Capacity'].astype(float) <= 130),'technology'] = 'Gas-OCGT' 

df_gen_agg_node = df_gen_2[df_gen_2['start_year']<=2015]
df_gen_agg_node = df_gen_agg_node.groupby(['node', 'technology'], as_index=False)['total_capacity'].sum()
df_gen_agg_node = df_gen_agg_node.pivot(index='node', columns='technology', values='total_capacity').fillna(0)

In [ ]:
df_gen_agg_node.to_csv(r'test_output_2.csv')